<a href="https://colab.research.google.com/github/Lukatariar/ECG_classify/blob/main/ECG_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import Counter
import re
from google.colab import drive
import os

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define the file path (update this to your actual file path in Google Drive)
file_path = "/content/drive/My Drive/MS_Thesis/ECG_Classify/ECG_data/20240319_P21_1  16_01_27_000 17_59_59_998.ascii"

# Read the file while skipping metadata lines
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Extract metadata lines for column naming
metadata_lines = [line for line in lines if line.startswith("# Col")]

# Parse column names from metadata
column_names = ["TIME"]  # First column is always TIME
column_counts = Counter()

for line in metadata_lines:
    parts = line.split(",")
    if len(parts) >= 4:
        col_name = parts[2].strip() + "_" + parts[3].strip()
        column_counts[col_name] += 1
        if column_counts[col_name] > 1:
            col_name = f"{col_name}_{column_counts[col_name]}"
        column_names.append(col_name)

# Identify the first non-metadata line (column headers)
data_start = next(i for i, line in enumerate(lines) if not line.startswith("#") and line.strip())

# Read data into a DataFrame
df = pd.read_csv(file_path, skiprows=data_start + 1, names=column_names, skipinitialspace=True)

# Display the parsed data
df.head()

<ipython-input-3-438c1e80a6a2>:28: DtypeWarning: Columns (1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=data_start + 1, names=column_names, skipinitialspace=True)


,TIME,ECG_mV,ECG_mV_2,ECG_mV_3,TEMP_Celsius,RAW_V,ACT_Counts,RAW_Days
0,3/19/2024 4:01:27.000 PM,-0.7672,-0.7446,-0.0562,35.9430,3.0885,856.4746,x
1,3/19/2024 4:01:27.002 PM,-0.7754,-0.7563,-0.0623,35.9430,3.0885,856.4746,x
2,3/19/2024 4:01:27.004 PM,-0.8019,-0.7650,-0.0584,35.9430,3.0885,856.4746,x
3,3/19/2024 4:01:27.006 PM,-0.8023,-0.7546,-0.0514,35.9430,3.0885,856.4746,x
4,3/19/2024 4:01:27.008 PM,-0.7884,-0.7346,-0.0484,35.9430,3.0885,856.4746,x


In [ ]:
# Convert TIME column to datetime
df["TIME"] = pd.to_datetime(df["TIME"], errors='coerce')

# Convert numeric columns (ignore errors for non-numeric values)
numeric_cols = df.columns[1:]  # Exclude TIME column
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [5]:
df.dtypes

,0
TIME,object
ECG_mV,object
ECG_mV_2,object
ECG_mV_3,object
TEMP_Celsius,object
RAW_V,object
ACT_Counts,object
RAW_Days,object
